In [26]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
import pandas, xgboost, numpy, textblob, string
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

In [27]:
sms = pd.read_csv("Spam.csv", encoding = 'latin-1')
print(sms)

      Class                                               Text
0     legit  Go until jurong point, crazy.. Available only ...
1     legit                      Ok lar... Joking wif u oni...
2      spam  Free entry in 2 a wkly comp to win FA Cup fina...
3     legit  U dun say so early hor... U c already then say...
4     legit  Nah I don't think he goes to usf, he lives aro...
5      spam  FreeMsg Hey there darling it's been 3 week's n...
6     legit  Even my brother is not like to speak with me. ...
7     legit  As per your request 'Melle Melle (Oru Minnamin...
8      spam  WINNER!! As a valued network customer you have...
9      spam  Had your mobile 11 months or more? U R entitle...
10    legit  I'm gonna be home soon and i don't want to tal...
11     spam  SIX chances to win CASH! From 100 to 20,000 po...
12     spam  URGENT! You have won a 1 week FREE membership ...
13    legit  I've been searching for the right words to tha...
14    legit                I HAVE A DATE ON SUNDAY WITH

In [28]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(sms['Text'], sms['Class'],  test_size=0.3)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [29]:
train_x.shape

(3900,)

In [30]:
##COUNT VECTORS AS FEATURES

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(sms['Text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [31]:
##NAIVE BAYES

naive_bayes = MultinomialNB().fit(xtrain_count, train_y)
predictions = naive_bayes.predict(xvalid_count)

print ("Test accuracy:",metrics.accuracy_score(predictions, valid_y))
print("Test CM",confusion_matrix(predictions, valid_y))

Test accuracy: 0.9838516746411483
Test CM [[1431   13]
 [  14  214]]


In [32]:
##LOGISTIC REGRESSION

logreg = linear_model.LogisticRegression().fit(xtrain_count, train_y)
predictions = logreg.predict(xvalid_count)

print ("Test accuracy:",metrics.accuracy_score(predictions, valid_y))
print("Test CM",confusion_matrix(predictions, valid_y))

Test accuracy: 0.9844497607655502
Test CM [[1444   25]
 [   1  202]]


C:\Users\10012191\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
rf = ensemble.RandomForestClassifier().fit(xtrain_count, train_y)
predictions = rf.predict(xvalid_count)

print ("Test accuracy:",metrics.accuracy_score(predictions, valid_y))
print("Test CM",confusion_matrix(predictions, valid_y))

C:\Users\10012191\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test accuracy: 0.9659090909090909
Test CM [[1443   55]
 [   2  172]]


In [34]:
xg = xgboost.XGBClassifier().fit(xtrain_count, train_y)
predictions = xg.predict(xvalid_count)

print ("Test accuracy:",metrics.accuracy_score(predictions, valid_y))
print("Test CM",confusion_matrix(predictions, valid_y))

Test accuracy: 0.972488038277512
Test CM [[1440   41]
 [   5  186]]


In [35]:
import pickle
with open ('NaiveBayes', 'wb') as c:
    pickle.dump(naive_bayes, c)
    
with open ('LogisticRegression', 'wb') as c:
    pickle.dump(naive_bayes, c)
    
with open ('RandomForest', 'wb') as c:
    pickle.dump(rf, c)
    
with open ('Xgboost', 'wb') as c:
    pickle.dump(xg, c)

In [36]:
# split the dataset into training and validation datasets 

sms = pd.read_csv("Spam.csv", encoding = 'latin-1')

valid_y = sms["Class"]
valid_x = sms["Text"]

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
valid_y = encoder.fit_transform(valid_y)

In [37]:
##COUNT VECTORS AS FEATURES

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(sms['Text'])

# transform the training and validation data using count vectorizer object
xvalid_count =  count_vect.transform(valid_x)

In [38]:
loaded_logistic_regression = pickle.load(open("NaiveBayes", "rb"))

predictions = loaded_logistic_regression.predict(xvalid_count)

print ("Test accuracy:",metrics.accuracy_score(predictions, valid_y))
print("Test CM",confusion_matrix(predictions, valid_y))


Test accuracy: 0.9908470926058865
Test CM [[4802   28]
 [  23  719]]


In [39]:
output_dataframe = pd.concat([sms, pd.DataFrame(predictions)], axis =1)

In [40]:
output_dataframe

,Class,Text,0
0,legit,"Go until jurong point, crazy.. Available only ...",0
1,legit,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,legit,U dun say so early hor... U c already then say...,0
4,legit,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,0
6,legit,Even my brother is not like to speak with me. ...,0
7,legit,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


In [41]:
import pickle
import sys

sys.path.append(r"C:\Users\10012191\Desktop\machinelearnclass\flash")
loaded_logistic_regression = pickle.load(open(r"NaiveBayes", "rb"))
